# Descargar el dataset desde kaggle

Para poder descargar el dataset usado en el modelo ejecuta el siguiente comando, ten en cuenta que estamos usando kaggle para esto y debes instalarlo y configurar la apiKey.

`kaggle datasets download -d grassknoted/asl-alphabet`

#### Mover Imagenes de test a train para hacer la división personalizada, asi cumplir con un 80 train / 20 test

In [1]:
import os
import shutil
import random
import math

In [ ]:
def move_test_images():
    """
    Moves test images to their corresponding letter folder in the train directory.
    """
    source_dir = '../data/raw/asl-alphabet/asl_alphabet_test/asl_alphabet_test'
    train_dir = '../data/raw/asl-alphabet/asl_alphabet_train/asl_alphabet_train'

    if not os.path.exists(source_dir):
        print(f"Error: Source directory not found at {source_dir}")
        return

    try:
        image_files = [f for f in os.listdir(source_dir) if f.endswith('.jpg')]
    except FileNotFoundError:
        print(f"Error: Could not list files in {source_dir}. It might not be a directory.")
        return


    for filename in image_files:
        letter = filename.split('_')[0]

        destination_folder = os.path.join(train_dir, letter)

        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(destination_folder, filename)

        try:
            shutil.move(source_path, destination_path)
            print(f"Moved {filename} to {destination_folder}")
        except FileNotFoundError:
            print(f"Error: Could not find {filename} to move.")
        except Exception as e:
            print(f"An error occurred while moving {filename}: {e}")


move_test_images()
print("Script finished.")


In [4]:
def split_data(source_dir, processed_dir, split_ratio=0.8):
    """
    Splits the data from source_dir into training and testing sets
    and saves them in processed_dir.

    Args:
        source_dir (str): The path to the directory containing the raw data,
                          with subdirectories for each class.
        processed_dir (str): The path to the directory where the processed
                             (split) data will be saved.
        split_ratio (float): The ratio of training data to the total data.
    """
    train_dir = os.path.join(processed_dir, 'train')
    test_dir = os.path.join(processed_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    if not os.path.exists(source_dir):
        print(f"Error: Source directory not found at {source_dir}")
        return

    for letter_folder in os.listdir(source_dir):
        letter_path = os.path.join(source_dir, letter_folder)
        if os.path.isdir(letter_path):
            train_letter_dir = os.path.join(train_dir, letter_folder)
            test_letter_dir = os.path.join(test_dir, letter_folder)
            os.makedirs(train_letter_dir, exist_ok=True)
            os.makedirs(test_letter_dir, exist_ok=True)

            images = [f for f in os.listdir(letter_path) if os.path.isfile(os.path.join(letter_path, f))]
            random.shuffle(images)

            split_point = math.ceil(len(images) * split_ratio)
            train_images = images[:split_point]
            test_images = images[split_point:]

            for image in train_images:
                source_image_path = os.path.join(letter_path, image)
                dest_image_path = os.path.join(train_letter_dir, image)
                shutil.copyfile(source_image_path, dest_image_path)
            
            print(f"Copied {len(train_images)} images to {train_letter_dir}")

            for image in test_images:
                source_image_path = os.path.join(letter_path, image)
                dest_image_path = os.path.join(test_letter_dir, image)
                shutil.copyfile(source_image_path, dest_image_path)

            print(f"Copied {len(test_images)} images to {test_letter_dir}")



SOURCE_DATA_DIR = '../data/raw/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
PROCESSED_DATA_DIR = '../data/processed'
    
split_data(SOURCE_DATA_DIR, PROCESSED_DATA_DIR)
print("Data splitting finished.")


Copied 2401 images to ../data/processed/train/R
Copied 600 images to ../data/processed/test/R
Copied 2401 images to ../data/processed/train/U
Copied 600 images to ../data/processed/test/U
Copied 2401 images to ../data/processed/train/I
Copied 600 images to ../data/processed/test/I
Copied 2401 images to ../data/processed/train/N
Copied 600 images to ../data/processed/test/N
Copied 2401 images to ../data/processed/train/G
Copied 600 images to ../data/processed/test/G
Copied 2401 images to ../data/processed/train/Z
Copied 600 images to ../data/processed/test/Z
Copied 2401 images to ../data/processed/train/T
Copied 600 images to ../data/processed/test/T
Copied 2401 images to ../data/processed/train/S
Copied 600 images to ../data/processed/test/S
Copied 2401 images to ../data/processed/train/A
Copied 600 images to ../data/processed/test/A
Copied 2401 images to ../data/processed/train/F
Copied 600 images to ../data/processed/test/F
Copied 2401 images to ../data/processed/train/O
Copied 600 i

# Normalización y preprocesamiento de datos
1. Normalizar
2. Convertir a escala de grises

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
TRAIN_DIR = '../data/processed/train'
TEST_DIR = '../data/processed/test'

# 1. Create a data generator for the training set with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 2. Create a data generator for the test set (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# 3. Create the actual generator objects from the directories
# They will generate batches of image data and labels indefinitely
BATCH_SIZE = 32
IMG_HEIGHT = 200 # The dataset images are 200x200
IMG_WIDTH = 200

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical' # for multi-class classification
)

validation_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# To see the class indices
print("Class indices:", train_generator.class_indices)


Found 69628 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
Class indices: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}
Found 17400 images belonging to 29 classes.
Class indices: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}


## Arquitectura de la red neuronal

In [13]:
model = keras.Sequential(
    [
        layers.Input(shape=(120000,)),
        layers.Dense(512, activation="relu"),
        layers.Dense(29, activation="softmax"),
    ]
)

model.summary()



Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 512)            │    61,440,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,455,389 (234.43 MB)

 Trainable params: 61,455,389 (234.43 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, InputLayer

# Get the number of classes from the generator
num_classes = len(train_generator.class_indices)

model = Sequential([
    # Define the input shape in the first layer
    InputLayer(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

    # First convolutional block
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Second convolutional block
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Third convolutional block
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Flatten the results to feed into a DNN
    Flatten(),

    # Dense layer for classification
    Dense(512, activation='relu'),

    # Output layer
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 97, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 46, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 67712)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │    34,669,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,777,181 (132.66 MB)

 Trainable params: 34,777,181 (132.66 MB)

 Non-trainable params: 0 (0.00 B)